In [2]:
from dataland_backend.models.data_type_enum import DataTypeEnum

from dataland_qa_lab.dataland.data_provider import get_yes_no_values_by_data
from dataland_qa_lab.pages.pages_provider import get_relevant_pages_of_pdf
from dataland_qa_lab.pages.text_to_doc_intelligence import extract_text_of_pdf
from dataland_qa_lab.review.yes_no_value_generator import extract_yes_no_template
from dataland_qa_lab.utils import config
from dataland_qa_lab.utils.nuclear_and_gas_data_collection import NuclearAndGasDataCollection

conf = config.get_config()
dataland_client = conf.dataland_client

In [3]:
company_infos = dataland_client.company_api.get_companies(data_types=[DataTypeEnum.NUCLEAR_MINUS_AND_MINUS_GAS])

company_ids = [company_info.company_id for company_info in company_infos]

print(len(company_ids))

data_ids = [
    dataland_client.eu_taxonomy_nuclear_and_gas_api.get_all_company_nuclear_and_gas_data(company_id=company_id)[
        0
    ].meta_info.data_id
    for company_id in company_ids
]

print(len(data_ids))

55
55


In [4]:
yes_no_values_dataland = {}
extracted_yes_no_values = {}

# check yes no values
for data_id, company_info in zip(data_ids[8:9], company_infos[8:9], strict=False):
    print(company_info.company_name)
    data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)
    data_collection = NuclearAndGasDataCollection(dataset=data.data)

    # get values on Dataland
    yes_no_values_dataland[company_info.company_name] = get_yes_no_values_by_data(data=data_collection)

    # get values from AI
    try:
        pdf_reader = get_relevant_pages_of_pdf(data_collection)
        text_of_page = extract_text_of_pdf(pdf_reader)
        extracted_yes_no_values[company_info.company_name] = extract_yes_no_template(text_of_page)
    except ValueError as e:
        # the support for multiple pages is not implemented yet
        print(f"Skipping company {company_info.company_name} due to invalid page numbers: {e}")

Concordia Versicherungs-Gesellschaft auf Gegenseitigkeit


ValueError: invalid literal for int() with base 10: '59-60'

In [30]:
total_sections_yes_no = 0
matching_sections_yes_no = 0

for company, dataland_dict in yes_no_values_dataland.items():
    print("\nCompany:", company)
    if company in extracted_yes_no_values:
        extracted_dict = extracted_yes_no_values[company]

        for section, value in dataland_dict.items():
            extracted_value = extracted_dict.get(section, "Not found")
            print(f"{section}: Dataland={value}, Extracted={extracted_value}")

            total_sections_yes_no += 1
            if value == extracted_value:
                matching_sections_yes_no += 1
    else:
        print(f"No extracted values found for company: {company}")

matching_ratio = matching_sections_yes_no / total_sections_yes_no
print(matching_ratio)


Company: ATLAS COPCO AKTIEBOLAG
nuclear_energy_related_activities_section426: Dataland=YesNo.NO, Extracted=YesNo.NO
nuclear_energy_related_activities_section427: Dataland=YesNo.NO, Extracted=YesNo.NO
nuclear_energy_related_activities_section428: Dataland=YesNo.NO, Extracted=YesNo.NO
fossil_gas_related_activities_section429: Dataland=YesNo.NO, Extracted=YesNo.NO
fossil_gas_related_activities_section430: Dataland=YesNo.NO, Extracted=YesNo.NO
fossil_gas_related_activities_section431: Dataland=YesNo.NO, Extracted=YesNo.NO

Company: BARCLAYS BANK IRELAND PUBLIC LIMITED COMPANY
nuclear_energy_related_activities_section426: Dataland=YesNo.YES, Extracted=YesNo.YES
nuclear_energy_related_activities_section427: Dataland=YesNo.YES, Extracted=YesNo.YES
nuclear_energy_related_activities_section428: Dataland=YesNo.YES, Extracted=YesNo.YES
fossil_gas_related_activities_section429: Dataland=YesNo.YES, Extracted=YesNo.NO
fossil_gas_related_activities_section430: Dataland=YesNo.YES, Extracted=YesNo.NO
